# experimentspydesign
## Demonstration
#### Josh McCrary

# Can we just show $\LaTeX$ in $\mathbb{Jupyter}$?
$$ \Omega = \sum_{j=1}^k \frac{ \prod_{i=1}^j \phi_{ij}\rho_{ij} }{\sum_{i=1}^k \psi_{ij} }$$

# Setup environment to visualize examples

In [1]:
import numpy as np
from bokeh.plotting import output_notebook, Figure, show
from bokeh.resources import INLINE
from bokeh.models import Column, ColumnDataSource, Range1d, Row, Circle, HoverTool, Div, Spacer

np.set_printoptions(suppress=True, precision=3)
output_notebook(resources=INLINE)

Loading BokehJS ...

# Define a convenience function for viewing designs

In [2]:
def scatter_matrix(design, k_factors=None, width=None, height=None, margin=2, size=8, alpha=1, all_range=None, source=None):
    if k_factors is None:
        k_factors = len(design[0, :]) if type(design) is np.ndarray else len(design)
        k_factors = k_factors if k_factors <= 10 else 10
    
    width = 1000 if width is None else width
    width = width // k_factors - margin
    height = 800 if height is None else height
    height = height // k_factors - margin
    
    if source is None or k_factors > 2:
        fsource = ColumnDataSource(data=dict([(str(i), design[:, i]) for i in range(k_factors)]))

    colors = np.arange(k_factors, k_factors * 14 // 5)
    colors = np.vstack([np.random.choice(colors, 3) for i in range(k_factors)]) / k_factors / 2
    
    grey = (234, 234, 234)
    sgrey = np.array(grey) / 255
    
    if k_factors == 2:
        color = (colors[0, :] * colors[1, :] * 128).astype(int)
        scolor = color / 255
        color = tuple(color)
        md = (scolor - sgrey).sum()
        light_color = tuple(((scolor ** 4 ** md) * 255).astype(int))
        g = Figure(width=width * 2, height=height * 2, tools='box_select,wheel_zoom', toolbar_location=None)
        if source is not None:
            glyph = g.circle(x=design[0], y=design[1], source=source, color=color, line_color=None, size=size, alpha=alpha)
            
            glyph.selection_glyph = Circle(fill_alpha=np.sqrt(alpha), fill_color=color, line_color=None)
            glyph.nonselection_glyph = Circle(fill_alpha=1, fill_color='#EAEAEA', line_color=light_color,
                                              line_alpha=1, line_width=size/4)
        else:
            g.circle(x=str(0), y=str(1), source=fsource, color=color, line_color=None, size=size, alpha=alpha)
        
        if all_range is not None:
            g.y_range = Range1d(*all_range)
            g.x_range = Range1d(*all_range)
        return g

    rows = []
    for i in range(k_factors):
        row = []
        for j in range(k_factors):
            #if i == j:
            #    continue
            if i <= j:
                color = (colors[i, :] * colors[j, :] * 128).astype(int)
                scolor = color / 255
                color = tuple(color)
                md = (scolor - sgrey).sum()
                light_color = tuple(((scolor ** 4 ** md) * 255).astype(int))
                g = Figure(width=width, height=height, tools='box_select,wheel_zoom', toolbar_location=None)
                glyph = g.circle(x=str(j), y=str(i), source=fsource, color=color, line_color=None, size=size, alpha=alpha)
                glyph.selection_glyph = Circle(fill_alpha=np.sqrt(alpha), fill_color=color, line_color=None)
                glyph.nonselection_glyph = Circle(fill_alpha=1, fill_color='#EAEAEA', line_color=light_color,
                                                  line_alpha=1, line_width=size/4)

                g.xaxis.visible = False
                g.yaxis.visible = False
                if all_range is not None:
                    g.y_range = Range1d(*all_range)
                    g.x_range = Range1d(*all_range)
                row.append(g)
            else:
                row.append(Spacer(width=width))
        rows.append(Row(*row))

    return Column(*rows)

def cc_v_bb_comparison(n=4, width=800, alpha=0.6, size=10, face='ccf'):
    w = (width - 40) // 2
    h = width // 2
    cc = ccdesign(n, face=face)
    all_range = (-1.5, 1.5) if face is 'ccf' else None
    cc_g = scatter_matrix(cc, size=size, height=h, width=w, alpha=alpha, all_range=all_range)

    bb = bbdesign(n)
    bb_g = scatter_matrix(bb, size=size, height=h, width=w, alpha=alpha,  all_range=(-1.5, 1.5))

    inner_text = ('<h1 style="text-align:center;">{}</h1>'
                  + '<h3 style="text-align:center;margin-top:5px;color:#7C7C7C;">{:d} Factors : {:d} Experiments</h3>')

    return Column(Row(Div(width=w, height=40, text=inner_text.format('Central-Composite', n, len(cc))),
                      Spacer(width=40),
                      Div(width=w, height=40, text=inner_text.format('Box-Behnken', n, len(bb)))),
                  Row(cc_g, Spacer(width=40), bb_g))

# experimentspydesign
## Main Features

Only depends on numpy

Factor classes use binary operaters to create factorial combinations

Contains simple design methods with selectable scaling

Factor classes expand scaling interface and enable more explicit and persistent definitions for particular collections of factors for a design

# Interface Overview
Provide the number of factors in the design (excluding fullfact) and define the default scale:

    method(number_of_factors, def_scale='traditional', **kwargs)

Submit an iterable or Factor class for each factor with the levels for the factor and/or an integer to define the number of levels:

    method((-10, 100), 7, np.arange(5, 14) **2, 
           def_scale='standard', **kwargs)
Same as:
    
    method([(-10, 100), 7, np.arange(5, 14) **2], 
           def_scale='standard', **kwargs)

Factors with explicit definitions will be scaled to their definitions and only factors with the number of levels defined will follow the `def_scale` paradigm.


# Import design methods

In [5]:
from experimentspydesign import ff2n, fullfact, bbdesign, ccdesign, lhs

# ccdesign
## Central Composite

Design used for quadratic models of a respone expands ff2n design by adding *star points on the hypersphere(ball) or hypercube of the design space.
Central Composite designs have $ 2^k + 2k + n_c $ experiments, where $n_c \geq 1$ is a number of additional points added at the origin for variance 

#### Circumscribed
The *star points which are added to the ff2n design are scaled outside of the factor's level range to maintain rotatability or some amount of orthogonality in the design. 

#### Inscribed
The *star points are placed on the center of the face of each side of the k-dimensional cube and the corners are scaled down to maintain rotatability or some amount of orthogonality in the design.

#### Face
The *star points are placed on the center of the face of each side of the k-dimensional cube and the corners retain their low and high values, this design is not rotatable nor orthogonal

In [7]:
ccc = ccdesign(3, 3, 3, face='ccc') # default face paradigm
cci = ccdesign(3, 3, 3, face='cci')
ccf = ccdesign(3, 3, 3, face='ccf')
print(len(ccc), 'experiments:')
print(ccc[:15, :])

19 experiments:
[[-1.    -1.    -1.   ]
 [ 1.    -1.    -1.   ]
 [-1.     1.    -1.   ]
 [ 1.     1.    -1.   ]
 [-1.    -1.     1.   ]
 [ 1.    -1.     1.   ]
 [-1.     1.     1.   ]
 [ 1.     1.     1.   ]
 [ 1.682  0.     0.   ]
 [ 0.     1.682  0.   ]
 [ 0.     0.     1.682]
 [-1.682  0.     0.   ]
 [ 0.    -1.682  0.   ]
 [ 0.     0.    -1.682]
 [ 0.     0.     0.   ]]


# bbdesign
## Box-Behnken

Similar to central composite and used for quadratic models, but more efficient for larger number of factors and maintains good rotatability.

In [8]:
bb = bbdesign(3, 3, 3, n_centers=4)
print(len(bb), 'experiments:')
print(bb[:15, :])

16 experiments:
[[-1 -1  0]
 [ 1 -1  0]
 [-1  1  0]
 [ 1  1  0]
 [-1  0 -1]
 [ 1  0 -1]
 [-1  0  1]
 [ 1  0  1]
 [ 0 -1 -1]
 [ 0  1 -1]
 [ 0 -1  1]
 [ 0  1  1]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]]


In [9]:
cc_v_bb = cc_v_bb_comparison(n=7, width=900, size=7, alpha=0.5)
show(cc_v_bb)

# bbdesign vs ccdesign

In [20]:
show(Column(Row(bb_12, bb_13, bb_23), Row(ccc_g, cci_g, ccf_g), f))

# lhs 
## Latin Hypercube Sampling
Choose the number of experiments in the design $ n $, each factor will have $ n $ levels evenly spaced from the low to high levels. 

If a factor has the same number of levels, $ n $, explicitly defined it will have those values in the final design rather than a linear spacing across the low to high values.

In [6]:
n = 11
factors = (1, 1, [(i - 6) * abs(i - 6) for i in range(n)]) if n == 10 else 3
des = lhs(factors, n_samples=n, def_scale='level_n')
print(len(des), 'experiments:')
print(des)

11 experiments:
[[ 8  7  2]
 [ 2  0  3]
 [ 1  3  1]
 [ 9  5  0]
 [ 4  2  5]
 [11  1 11]
 [ 0  4  8]
 [ 6  9  6]
 [ 3  8  7]
 [ 5 11  4]
 [ 7  6  9]]


## Select a range in a factor and see how those experiments are distributed in other factors

In [11]:
des = lhs([2] * 7, n_samples=149)
des_g = scatter_matrix(des, size=6, height=600, width=800)
show(des_g)